In [ ]:
# cài ultralytic
!pip install ultralytics
!pip install deepface

In [3]:
from ultralytics import YOLO
import json
import os
from deepface import DeepFace
import cv2

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
25-05-19 17:15:45 - Directory /root/.deepface has been created
25-05-19 17:15:45 - Directory /root/.deepface/weights has been created


In [24]:
class FaceDatabase:
    def __init__(self, json_file_path,path_weight_yolo):
        self.json_file_path = json_file_path
        # Kiểm tra xem file JSON có tồn tại không, nếu không sẽ tạo mới
        if os.path.exists(self.json_file_path):
            with open(self.json_file_path, 'r') as f:
                self.images_data = json.load(f)
        else:
            self.images_data = []  # Nếu file chưa có, tạo danh sách rỗng
        self.model_detect_face = YOLO(path_weight_yolo)

    def crop_face(self,result):
      a = result[0].boxes.xyxy.tolist()
      if len(a)==0:
        return None
      img = result[0].orig_img
      x0,y0,x1,y1 = result[0].boxes.xyxy.tolist()[0]
      crop_img = img[int(y0):int(y1), int(x0):int(x1)]
      # convert RGB
      # crop_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)
      return crop_img

    def save_data(self):
        """Lưu dữ liệu vào file JSON"""
        with open(self.json_file_path, 'w') as f:
            json.dump(self.images_data, f, indent=4)

    def add_image(self, image_name, image_path):
        """Thêm ảnh mới vào database với image_id tự động tăng"""
        # Tạo ID tự động
        image_id = len(self.images_data) + 1  # Tạo ID mới tự động

        image = {
            'id': image_id,
            'name': image_name,
            'path': image_path
        }

        self.images_data.append(image)
        self.save_data()

        return f"Ảnh với ID {image_id} đã được thêm vào database."

    def edit_image(self, image_id, new_name=None, new_path=None):
        """Sửa thông tin ảnh theo ID"""
        for image in self.images_data:
            if image['id'] == image_id:
                if new_name:
                    image['name'] = new_name
                if new_path:
                    image['path'] = new_path
                self.save_data()
                return f"Đã sửa ảnh với ID {image_id}"
        return f"Không tìm thấy ảnh với ID {image_id}"

    def delete_image(self, image_id):
      """Xóa ảnh theo ID"""
      # Kiểm tra xem ảnh có tồn tại trong danh sách hay không
      image_to_delete = None
      for image in self.images_data:
          if image['id'] == image_id:
              image_to_delete = image
              break

      if image_to_delete:
          # Xóa ảnh nếu tìm thấy
          self.images_data = [image for image in self.images_data if image['id'] != image_id]
          self.save_data()  # Lưu lại dữ liệu vào file JSON
          print(f"Đã xóa ảnh với ID {image_id}.")
      else:
          # Nếu không tìm thấy ảnh với ID
          print(f"Không tìm thấy ảnh với ID {image_id}.")


    def check_and_store_image(self, test_image_path):
        """Kiểm tra ảnh và lưu vào database nếu là ảnh đúng"""
        # Cắt khuôn mặt từ ảnh kiểm tra
        result = self.model_detect_face(test_image_path)  # Giả sử đây là hàm phát hiện khuôn mặt
        crop_img = self.crop_face(result)

        if crop_img is None:
            print("Không phát hiện mặt trong ảnh!")
            return None
        is_img = True
        if isinstance(test_image_path, str):
        # Lưu ảnh khuôn mặt đã cắt vào tệp tạm thời
            test_image_path = "/content/tmp_face.jpg"
            cv2.imwrite(test_image_path, crop_img)
        else:
            is_img = False

        # Kiểm tra ảnh và lưu vào database nếu là ảnh đúng
        closest_match = None
        highest_similarity = float('inf')  # Để so sánh và tìm ảnh giống nhất
        flag = False

        # Duyệt qua tất cả ảnh trong database (dựa vào 'path' trong mỗi phần tử JSON)
        for image in self.images_data:
            dataset_image_path = image['path']  # Lấy đường dẫn ảnh từ JSON

            # Sử dụng DeepFace để nhận diện và so sánh khuôn mặt
            try:
                if is_img == False:
                  result = DeepFace.verify(crop_img, dataset_image_path)
                  similarity = result['distance']
                else:
                  result = DeepFace.verify(test_image_path, dataset_image_path)
                  similarity = result['distance']

                # Kiểm tra xem khuôn mặt có được xác nhận là giống nhau hay không
                if result['verified'] == True and similarity < 0.6: #
                    print(f"Điểm danh thành công: {image['name']} với độ tương đồng: {similarity}")
                    flag = True
                    closest_match = image['name']
                    break
            except Exception as e:
                print(f"Lỗi khi so sánh ảnh {dataset_image_path} với ảnh kiểm tra: {e}")

        if not flag:
            print("Không điểm danh được")
# # # Sử dụng lớp FaceDatabase
json_file_path = '/content/images_data.json'
path_face_model ='/content/best.pt'
# # # Tạo đối tượng FaceDatabase
face_db = FaceDatabase(json_file_path,path_face_model)


## Test thử Chức năng ( Đổi lại đường dẫn theo máy mik để sử dụng)

In [32]:
img = '/content/001_08194468.jpg'
img = cv2.imread(img)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [33]:
# ảnh đã đọc, hoặc lấy từ camera,hoặc đường đã ảnh
face_db.check_and_store_image(img)


0: 640x640 1 face, 130.6ms
Speed: 5.9ms preprocess, 130.6ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
Điểm danh thành công: image8.jpg với độ tương đồng: 0.04043497593360745


In [26]:
face_db.add_image('image8.jpg', '/content/001_08194468.jpg')

'Ảnh với ID 8 đã được thêm vào database.'

In [34]:
face_db.edit_image(8, new_name='image8_updated.jpg', new_path='/content/001_08194468.jpg')


'Đã sửa ảnh với ID 8'

In [35]:
face_db.delete_image(8)

Đã xóa ảnh với ID 8.
